In [1]:
%matplotlib inline

In [2]:
from __future__ import division, print_function, absolute_import
import os
import sys
sys.path.append('/glade/work/yifanc/code/optmz/MO-ASMO/src')
import MOASMO
import numpy as np
import matplotlib.pyplot as plt
import util
import _pickle as cPickle
import pandas as pd
import sampling
import xarray as xr
from os import path
import os
import seaborn as sns

In [3]:
import GLP

In [4]:
namelist_file = 'data/lnd_in'
basepftfile = 'data/clm50_params.c210607.nc'
# basepftfile = '/glade/work/yifanc/NNA/optmz/paramfile/default_OM/clm50_params.c210507.OM.cdf5.nc'

In [5]:
f = open(namelist_file, "r")
namelist_f = pd.DataFrame([],columns=['value'])
for x in f:
    string_list = x.split(sep='=')
    st_list = []
    for st in string_list:
        st_list.append(st.strip())
    if len(st_list) == 2:
        df = pd.DataFrame([st_list[1]],columns=['value'],index=[st_list[0]])
        namelist_f = pd.concat([namelist_f,df],axis=0)

In [6]:
param_ds = xr.open_dataset(basepftfile)

# veggie type
Mainly veggie types 2, 11, and 12 are the major veggie types in the arctic, so I would only change the parameters for those three parameters

In [7]:
# load parameter name and range
# When in STEP 2: load this file
pf = pd.read_csv("data/selected_parameters_from.PPE.csv")# pf.parse('T14')
# # when in STEP 3: load this file
# pf = pd.read_csv("data/final_list_params_asmo.csv")

# starting from experiment T5, we need to filter parameters that was only used in fates
#pf = pf[pf['whether_asmo']=='y']
pf = pf[(pf['whether_asmo']=='y')&(pf['only_in_fates']=='no')]
# pf = pd.read_csv('/glade/work/yifanc/NNA/optmz/workflow/param_pertubed.t1.210422.txt',sep='\t')
nInput = len(pf)
nOutput = 2
file_location = pf['location'].values
N_var_list = pf[pf['location']=='N']['Parameters'].values
P_var_list = pf[pf['location']=='P']['Parameters'].values

# parameters for MO-ASMO
niter = 5
pct = 0.2

In [9]:
var_min='min_asmo'
var_max='max_asmo'

var_pft_min = 'pft_mins'
var_pft_max = 'pft_maxs'

var_param = 'Parameters'

pf[var_min] = pf[var_min].astype(str)
pf[var_max] = pf[var_max].astype(str)

pft_type_sel = np.arange(79)#np.array([2,11,12]).astype(np.int32)

In [10]:
vmin_list = []
vmax_list = []
for row in pf.iterrows():
    param_name = row[1][var_param]
    if row[1]['location'] == 'N':
        default_value = namelist_f.loc[param_name]
        if type(default_value) is str:
            default_value = float(default_value.split('d')[0])
        if 'percent' in row[1][var_min]:
            pct_min = float(row[1][var_min].split('percent')[0])
            pct_max = float(row[1][var_max].split('percent')[0])
            vmin_list.append(default_value*(1-pct_min/100))
            vmax_list.append(default_value*(1+pct_max/100))
        else:
            vmin_list.append(float(row[1][var_min]))
            vmax_list.append(float(row[1][var_max]))
    else:
        if row[1]['location'] == 'P':
            default_value = param_ds[param_name].values
            if 'percent' in row[1][var_min]:
                pct_min = float(row[1][var_min].split('percent')[0])
                pct_max = float(row[1][var_max].split('percent')[0])
                vmin_list.append(default_value*(1-pct_min/100))
                vmax_list.append(default_value*(1+pct_max/100))
            else:
                if row[1][var_min] == 'pft':
                    pft_mins = row[1][var_pft_min]
                    pft_maxs = row[1][var_pft_max]
                    pft_mins = np.array(pft_mins.split(',')).astype(np.float64)
                    pft_maxs = np.array(pft_maxs.split(',')).astype(np.float64)
                    vmin_list.append(pft_mins[pft_type_sel])
                    vmax_list.append(pft_maxs[pft_type_sel])
#                     print(pft_mins[pft_type_sel], pft_maxs[pft_type_sel])
                else:
                    if type(default_value) == np.ndarray:
                        vmin_list.append(np.full(default_value.shape,float(row[1][var_min])))
                        vmax_list.append(np.full(default_value.shape,float(row[1][var_max])))
                    else:
                        vmin_list.append(float(row[1][var_min]))
                        vmax_list.append(float(row[1][var_max]))
            print(param_name,vmin_list[-1])        
pf['vmin_value'] = vmin_list
pf['vmax_value'] = vmax_list

xlb = pf['vmin_value'].values
xub = pf['vmax_value'].values


fff 0.01
medlynslope [9.   1.53 1.53 1.53 1.7  1.7  2.05 2.05 2.05 0.91 0.91 0.91 3.78 3.78
 0.65 3.89 3.89 0.65 0.65 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89
 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89
 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89
 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 3.89 0.65 0.65 3.89
 3.89 0.65 0.65 0.65 0.65 0.65 0.65 3.89 3.89]
sand_pf -20.0
zsno 1e-05
d_max 5.0
e_ice 1.0
dleaf [0.       0.000216 0.000216 0.00072  0.0081   0.0081   0.0081   0.0081
 0.0081   0.0081   0.000405 0.000162 0.000144 0.000144 0.000144 0.000162
 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162
 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162
 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162
 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162
 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162 0.000162
 0.000162 0.00016

In [27]:
# vmax_list

In [11]:
pop = 100
Xinit = None
Yinit = None

In [12]:
"""
Multi-Objective Adaptive Surrogate Modelling-based Optimization
model: the evaluated model function
nInput: number of model input
nOutput: number of output objectives
xlb: lower bound of input
xub: upper bound of input
niter: number of iteration
pct: percentage of resampled points in each iteration
Xinit and Yinit: initial samplers for surrogate model construction
### options for the embedded NSGA-II of MO-ASMO
    pop: number of population
    gen: number of generation
    crossover_rate: ratio of crossover in each generation
    mu: distribution index for crossover
    mum: distribution index for mutation
"""


'\nMulti-Objective Adaptive Surrogate Modelling-based Optimization\nmodel: the evaluated model function\nnInput: number of model input\nnOutput: number of output objectives\nxlb: lower bound of input\nxub: upper bound of input\nniter: number of iteration\npct: percentage of resampled points in each iteration\nXinit and Yinit: initial samplers for surrogate model construction\n### options for the embedded NSGA-II of MO-ASMO\n    pop: number of population\n    gen: number of generation\n    crossover_rate: ratio of crossover in each generation\n    mu: distribution index for crossover\n    mum: distribution index for mutation\n'

In [13]:
N_resample = int(pop*pct)
Ninit = 200 #
Xinit = sampling.glp(Ninit, nInput)


In [15]:
np.shape(Xinit)

(200, 40)

In [16]:
perturbed_param = []
for i in range(Ninit):
    perturbed_param.append(Xinit[i,:] * (xub - xlb) + xlb)

perturbed_param = np.array(perturbed_param)

In [33]:
# perturbed_param

In [17]:
# create test id
location="pe_basin"
iteration=0

test_id_list=[]
for id_ in range(Ninit):
    test_id = '%s_%s_%s'%(location, "%i"%(iteration), "%04i"%(id_))
    test_id_list.append(test_id)

psets_df = pd.DataFrame(perturbed_param, columns=pf[var_param].values, index=test_id_list)

In [18]:
print("Variables that need to be changed in the Namelist file: %s"%(N_var_list))
print("Variables that need to be changed in the Parameter file: %s"%(P_var_list))
len(N_var_list), len(P_var_list)

Variables that need to be changed in the Namelist file: ['upplim_destruct_metamorph' 'maximum_leaf_wetted_fraction' 'jmaxb1'
 'baseflow_scalar']
Variables that need to be changed in the Parameter file: ['fff' 'medlynslope' 'sand_pf' 'zsno' 'd_max' 'e_ice' 'dleaf' 'slopebeta'
 'frac_sat_soil_dsl_init' 'jmaxb0' 'accum_factor' 'medlynintercept'
 'leafcn' 'tpu25ratio' 'cv' 'wc2wjb0' 'clay_pf' 'nstem' 'n_melt_coef'
 'xdrdt' 'a_exp' 'om_frac_sf' 'snw_rds_refrz' 'liq_canopy_storage_scalar'
 'z0mr' 'tkd_sand' 'C2_liq_Brun89' 'krmax' 'theta_cj' 'scvng_fct_mlt_sf'
 'slatop' 'wind_min' 'a_coef' 'ssi' 'vcmaxha' 'kmax']


(4, 36)

In [29]:
# psets_df['medlynslope']

In [19]:
psets_N_df = psets_df[N_var_list]
psets_P_df = psets_df[P_var_list]

In [23]:
psets_df.to_csv('workflow/%s_%s.param_list.txt'%(location, "%i"%(iteration)))

In [24]:
main_run = 'workflow/%s_%s.main_run.txt'%(location, "%i"%(iteration))
with open(main_run,"w") as file:
    case_list = '\n'.join(psets_df.index.values)
    case_list += '\n'
    file.write(case_list)
file.close()


In [35]:
# assign the basepftfile
# basepftfile = "../basecase/clm5_params.c200519.nc"
# basepftfile = '/glade/p/cesmdata/cseg/inputdata/lnd/clm2/paramdata/clm50_params.c210217.nc'

param_output_dir = "./paramfile/%s/"%(location)

if not path.isdir(param_output_dir):
    os.mkdir(param_output_dir)

# determine whether we need to generate new parameter files
if len(P_var_list)>0:
        
    for index, row in psets_P_df.loc[['pe_basin_0_0123']].iterrows():#psets_P_df.iterrows():
        
        # open the default file
        tmp = xr.open_dataset(basepftfile)
        
        pftfile = param_output_dir+index+".nc"
        
        for var in P_var_list:
            
            if pf[pf[var_param]==var][var_min].values[0] == 'pft':
                var_value = tmp[var].values
                if len(var_value.shape) == 1:
                    var_value[pft_type_sel] = row[var]
                else:
                    var_value[:,pft_type_sel] = row[var]
                tmp[var] = xr.DataArray(var_value,dims=tmp[var].dims)
                
            else:
                if type(row[var]) == np.ndarray:
                    dims = tmp[var].dims
                    tmp[var] = xr.DataArray(row[var], dims=dims)
#                     print(var,row[var])
                else:
                    tmp[var] = row[var]
            
        print('Done working on '+pftfile)

        tmp.to_netcdf(pftfile,'w')

Done working on /glade/work/yifanc/NNA/optmz/paramfile/pe_basin/pe_basin_0_0123.nc


In [31]:
# psets_N_df

In [28]:
namelist_dir='./namelist/%s/'%(location)

if not path.isdir(namelist_dir):
    os.mkdir(namelist_dir)

# determine whether we need to generate new namelist files
# if len(N_var_list)>0:
    
for index, row in psets_N_df.iterrows():
    nlfile = namelist_dir+index+".txt"
    with open(nlfile,"w") as file:
        output = "! user_nl_clm namelist options written by generate_params:\n"
        file.write(output)

for index, row in psets_N_df.iterrows():
    nlfile = namelist_dir+index+".txt"
    print('working on '+nlfile)
    for var in N_var_list:
        with open(nlfile,"a") as file: # key is using "a" for append option
            print(var+' modified')
            output = "%s=%s\n" % (var, float(row[var])) #round??
            file.write(output) 


working on /glade/work/yifanc/NNA/optmz/namelist/pe_basin/pe_basin_0_0000.txt
upplim_destruct_metamorph modified
working on /glade/work/yifanc/NNA/optmz/namelist/pe_basin/pe_basin_0_0001.txt
upplim_destruct_metamorph modified
working on /glade/work/yifanc/NNA/optmz/namelist/pe_basin/pe_basin_0_0002.txt
upplim_destruct_metamorph modified
working on /glade/work/yifanc/NNA/optmz/namelist/pe_basin/pe_basin_0_0003.txt
upplim_destruct_metamorph modified
working on /glade/work/yifanc/NNA/optmz/namelist/pe_basin/pe_basin_0_0004.txt
upplim_destruct_metamorph modified
working on /glade/work/yifanc/NNA/optmz/namelist/pe_basin/pe_basin_0_0005.txt
upplim_destruct_metamorph modified
working on /glade/work/yifanc/NNA/optmz/namelist/pe_basin/pe_basin_0_0006.txt
upplim_destruct_metamorph modified
working on /glade/work/yifanc/NNA/optmz/namelist/pe_basin/pe_basin_0_0007.txt
upplim_destruct_metamorph modified
working on /glade/work/yifanc/NNA/optmz/namelist/pe_basin/pe_basin_0_0008.txt
upplim_destruct_me